In [52]:
import torch

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-05-07 12:51:28--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.27MB/s    in 0.3s    

2023-05-07 12:51:30 (3.27 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [358]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [359]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [360]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [361]:
token_list = sorted(list(set(text)))
vocab_size = len(token_list)
print(len(token_list), token_list)

65 ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [362]:
# Every token needs to be encoded into its int value
stoi = {ch:i for i, ch in enumerate(textList)}
itos = {i:ch for ch, i in stoi.items()}

In [363]:
# A given block of tokens needs to encoded into a list of ints

def encode(text_block):
    encoded_text = []
    for ch in text_block:
        encoded_text.append(stoi[ch])
    return encoded_text

def decode(encoded_text):
    decoded_text = []
    for i in encoded_text:
        decoded_text.append(itos[i])
    return ''.join(decoded_text)

encoded_text = encode(text)

In [364]:
# Need to split the dataset into train 80%, dev 10%, test sets 10%
time_series = block_size = 8

xtrain, ytrain = [], []
xDev, yDev = [], []
xTest, yTest = [], []

def get_data_split(split_type):
    train_len = int(0.8*len(encoded_text))
    val_len = int(0.1*len(encoded_text)) + train_len
    
    if split_type == "train":
        x = encoded_text[:train_len]
    elif split == "val":
        x = encoded_text[train_len:val_len]
    else:
        x = encoded_text[val_len:]
        
    return torch.tensor(x)

train_data = get_data_split("train")
val_data = get_data_split("val")
test_data = get_data_split("test")
train_data.shape

torch.Size([892315])

In [399]:
# Function to get one batch at a time
train_batch_size = B = 4
val_batch_size = 4

def get_batch(split):
    if split == "train": 
        batch_size = train_batch_size
        data = train_data
    elif split == "val": 
        batch_size = val_batch_size
        data = val_data
    else: 
        batch_size = test_batch_size
        data = test_data
#     print(len(data_split))
    indexes = torch.randint(len(data)-time_series, (batch_size,))
#     print(indexes.shape)
    batch_x = torch.stack([data[i: block_size+i] for i in indexes])#.to(torch.float32)
    batch_y = torch.stack([data[i+1: block_size+i+1] for i in indexes])#.to(torch.float32)
    return batch_x, batch_y


x, y = get_batch("train")
x.shape, x        

(torch.Size([4, 8]),
 tensor([[50, 39, 52, 58, 43, 56, 52,  6],
         [63,  1, 42, 53,  1, 63, 53, 59],
         [53, 56,  1, 42, 53,  1, 21,  1],
         [ 1, 42, 43, 57, 54, 39, 47, 56]]))

In [383]:
channel_size = 16
no_blocks = 5
no_heads = 4
head_size = 16

class BigramLanguageModel(torch.nn.Module):
    def __init__(self, time_series, channel_size):
        super().__init__()
        self.token_embed = Linear(vocab_size, channel_size)
        self.positional_embed = Linear(vocab_size, channel_size)
        self.blocks = torch.nn.Sequential(*[Block(time_series, channel_size, head_size)]*no_blocks)
        self.layer_norm = LayerNorm(time_series)
        self.linear = Linear(channel_size, vocab_size)
        
    def forward(self, x, y=None):
        x = self.token_embed.W[x] + self.positional_embed.W[x]
#         x = self.token_embed(x) + self.positional_embed(x)
        x = self.blocks(x)
        x = self.layer_norm(x)
        logits = self.linear(x)
        
        if y != None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = torch.nn.functional.cross_entropy(logits, y)
            return loss

In [384]:
class Linear(torch.nn.Module):
    def __init__(self, fan_in, fan_out, bias=False):
        super().__init__()
        self.W = torch.nn.Parameter(torch.randn((fan_in, fan_out)) / (fan_in**0.5))
        if bias: self.B = torch.nn.Parameter(torch.randn((fan_out,)))
        else: self.B = None
            
    def forward(self, x):
        out = x @ self.W
        if self.B != None: out += self.B
            
        return out
            
linear = Linear(3, 5, True)
x = torch.randn((2, 3))
# print(linear.B, linear.B.shape)
print(linear(x))

tensor([[-1.5979,  1.3380,  0.7685, -0.7797,  0.0387],
        [ 0.0468,  1.7423,  1.3627, -1.4605, -1.2071]], grad_fn=<AddBackward0>)


In [385]:
class LayerNorm(torch.nn.Module):
    def __init__(self, time_series):
        super().__init__()
        self.gamma = torch.randn((time_series, 1))
        self.beta = torch.randn((time_series, 1))
        self.epsil = 0.0001
        
    def forward(self, x): # x -> (B, T, C)
        mean = torch.mean(x, dim=2, keepdim=True) # (B, T, 1)
        var = torch.var(x, dim=2, keepdim=True)
        
        out = self.gamma*((x-mean)/(var+self.epsil)**0.5) + self.beta
        return out

layer = LayerNorm(time_series)
x = torch.randn((B, time_series, channel_size))
mean = layer(x)
mean.shape, x.shape

(torch.Size([1, 8, 16]), torch.Size([1, 8, 16]))

In [386]:
class FeedForward(torch.nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.net = torch.nn.Sequential(
            Linear(head_size, 4*head_size, bias=True),
            Relu(),
            Linear(4*head_size, channel_size)
#             torch.nn.Dropout(dropout)
        )
        
    def forward(self, x):
        return self.net(x)

In [387]:
class Relu(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        return self.relu(x)

In [388]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, T, C, head_size):
        super().__init__()
#         self.no_heads = 4
        self.heads = torch.nn.ModuleList([Head(T, C, head_size//4) for _ in range(4)])
    
    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=2) # head(x) -> (B, T, H//4)
        return out
        

In [389]:
class Head(torch.nn.Module):
    def __init__(self, T, C, head_size):
        super().__init__()
        self.query = Linear(C, head_size)
        self.key = Linear(C, head_size)
        self.value = Linear(C, head_size)
        self.register_buffer("tril", torch.tril(torch.ones(T, T)))
        
    def forward(self, x): # x -> (B, T, C)
        query = self.query(x)
        key = self.key(x) 
        value = self.value(x) # (B, T, H)
        
        wei = query @ key.transpose(-2, -1) # (B, T, T)
        wei = wei.masked_fill(self.tril == 0, -float("inf"))
        wei = torch.softmax(wei, dim=2)
        
        return wei @ value # (B, T, H)

In [390]:
class Block(torch.nn.Module):
    def __init__(self, T, C, head_size):
        super().__init__()
        self.layers = torch.nn.Sequential(
            LayerNorm(T),
            MultiHeadAttention(T, C, head_size),
            LayerNorm(T),
            FeedForward(head_size)
        )
        
    def forward(self, x):
        return self.layers(x) 

In [391]:
B, C, T, H = 1, 4, 5, 6
head = Head(T, C, H)
x = torch.randn((B, T, C))
head.forward(x)
params = head.parameters()
next(params)

Parameter containing:
tensor([[ 0.3818,  0.0725, -0.0870, -0.4300,  0.5514, -0.2411],
        [ 0.2201,  0.9559, -0.5193, -0.5000, -0.3703,  0.2533],
        [ 0.6238,  1.3190,  0.4906, -0.1029, -0.1468,  0.1250],
        [ 0.2720,  0.3514, -0.4024, -0.6464, -0.2954,  0.8430]],
       requires_grad=True)

In [401]:
@torch.no_grad()
def estimate_loss():
    out = {}
#     model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
#     model.train()
    return out

model = BigramLanguageModel(time_series, channel_size)
# # print(next(model.parameters()))
# batch_size = 16 # how many independent sequences will we process in parallel?
# block_size = time_series = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# eval_iters = 200
# n_embd = 64
# n_head = 4
# n_layer = 4

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

#     every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


# for _ in range(100):
#     x, y = get_batch("train")
#     loss = model(x, y)
#     print(loss)
#     break

step 0: train loss 5.1009, val loss 5.0929
step 100: train loss 4.2075, val loss 4.1723
step 200: train loss 3.8840, val loss 3.8977
step 300: train loss 3.5858, val loss 3.6317
step 400: train loss 3.4831, val loss 3.5293
step 500: train loss 3.4288, val loss 3.4714
step 600: train loss 3.4067, val loss 3.4557
step 700: train loss 3.4110, val loss 3.4149
step 800: train loss 3.3788, val loss 3.4067
step 900: train loss 3.3742, val loss 3.4147
step 1000: train loss 3.3781, val loss 3.4077
step 1100: train loss 3.3740, val loss 3.4162
step 1200: train loss 3.3666, val loss 3.4391
step 1300: train loss 3.3999, val loss 3.3806
step 1400: train loss 3.3276, val loss 3.3868
step 1500: train loss 3.3720, val loss 3.3986
step 1600: train loss 3.3147, val loss 3.3794
step 1700: train loss 3.3334, val loss 3.3776
step 1800: train loss 3.3210, val loss 3.3693
step 1900: train loss 3.3409, val loss 3.4247
step 2000: train loss 3.3415, val loss 3.3677
step 2100: train loss 3.3133, val loss 3.3752


In [267]:
arr = [*[i for i in range(5)]]
arr

[0, 1, 2, 3, 4]

In [283]:
t = torch.randn((4, 5))
x = torch.randint(4, (4, 5))
print(x)
t[x]

tensor([[3, 2, 0, 2, 0],
        [1, 0, 3, 1, 3],
        [1, 1, 1, 0, 3],
        [2, 2, 2, 1, 3]])


tensor([[[ 0.2421,  0.0626, -0.9244, -0.5468, -1.2528],
         [ 2.2814,  2.0784, -0.8457, -0.8192,  0.2828],
         [ 1.4330,  0.0588,  3.0397,  0.0722, -0.4317],
         [ 2.2814,  2.0784, -0.8457, -0.8192,  0.2828],
         [ 1.4330,  0.0588,  3.0397,  0.0722, -0.4317]],

        [[-1.0457, -1.0525,  0.7554, -1.0383,  1.9414],
         [ 1.4330,  0.0588,  3.0397,  0.0722, -0.4317],
         [ 0.2421,  0.0626, -0.9244, -0.5468, -1.2528],
         [-1.0457, -1.0525,  0.7554, -1.0383,  1.9414],
         [ 0.2421,  0.0626, -0.9244, -0.5468, -1.2528]],

        [[-1.0457, -1.0525,  0.7554, -1.0383,  1.9414],
         [-1.0457, -1.0525,  0.7554, -1.0383,  1.9414],
         [-1.0457, -1.0525,  0.7554, -1.0383,  1.9414],
         [ 1.4330,  0.0588,  3.0397,  0.0722, -0.4317],
         [ 0.2421,  0.0626, -0.9244, -0.5468, -1.2528]],

        [[ 2.2814,  2.0784, -0.8457, -0.8192,  0.2828],
         [ 2.2814,  2.0784, -0.8457, -0.8192,  0.2828],
         [ 2.2814,  2.0784, -0.8457, -0.81

In [407]:
conv = "Disney+ Targeting Geo (country)|Disney+ Language (Auto-tag)|Disney+ Brand (Auto-tag)|Disney+ Title (Auto-tag)|Disney+ Campaign Objective (Auto-tag)|Disney+ Content Type (Auto-tag)|Disney+ Quarter / Timing (Auto-tag)|Disney+ Targeting Strategy (Auto-tag)|Disney+ Targeting Type (Auto-tag)|Disney+ Audience Detail (Auto-tag)|Disney+ Platform/Vendor Name (Auto-tag)|Disney+ Placement (Auto-tag)|Disney+ Device (Auto-tag)|Disney+ Ad Format (Auto-tag)|Disney+ Unit Size (Auto-tag)|Disney+ Channel (Internal)|Disney+ Creative (Auto-tag)|Disney+ Detail FreeForm (Auto-tag)|Disney+ Campaign Owner (Auto-tag)|Disney+ Product (Auto-tag)|Disney+ Geo Cluster (Auto-tag)|Disney+ Placement/Ad ID (Auto-tag)"
convList = list(enumerate(conv.split("|")))
convList, len(convList)

([(0, 'Disney+ Targeting Geo (country)'),
  (1, 'Disney+ Language (Auto-tag)'),
  (2, 'Disney+ Brand (Auto-tag)'),
  (3, 'Disney+ Title (Auto-tag)'),
  (4, 'Disney+ Campaign Objective (Auto-tag)'),
  (5, 'Disney+ Content Type (Auto-tag)'),
  (6, 'Disney+ Quarter / Timing (Auto-tag)'),
  (7, 'Disney+ Targeting Strategy (Auto-tag)'),
  (8, 'Disney+ Targeting Type (Auto-tag)'),
  (9, 'Disney+ Audience Detail (Auto-tag)'),
  (10, 'Disney+ Platform/Vendor Name (Auto-tag)'),
  (11, 'Disney+ Placement (Auto-tag)'),
  (12, 'Disney+ Device (Auto-tag)'),
  (13, 'Disney+ Ad Format (Auto-tag)'),
  (14, 'Disney+ Unit Size (Auto-tag)'),
  (15, 'Disney+ Channel (Internal)'),
  (16, 'Disney+ Creative (Auto-tag)'),
  (17, 'Disney+ Detail FreeForm (Auto-tag)'),
  (18, 'Disney+ Campaign Owner (Auto-tag)'),
  (19, 'Disney+ Product (Auto-tag)'),
  (20, 'Disney+ Geo Cluster (Auto-tag)'),
  (21, 'Disney+ Placement/Ad ID (Auto-tag)')],
 22)

In [441]:
name = "KR|KO|Multi|Rotation|Consid|Rotation|23'Q3|Prospecting|Behavior|Interest|TikTok|Feed|Android|Multi|Multi|Social|NA|Android|DSS|Disney+|KR|"
# name = "CA|EN|Multi|Rotation|Acq|Commerce|23'Q3|Prospecting|Demo|A18+|FBIG|Feed|CrossDev|Multi|Multi|Social|NA|CommerceCatalog|DSS|Disney+|CA|366389807"
nameList = list(enumerate(name.split("|")))
nameList, len(nameList)

([(0, 'KR'),
  (1, 'KO'),
  (2, 'Multi'),
  (3, 'Rotation'),
  (4, 'Consid'),
  (5, 'Rotation'),
  (6, "23'Q3"),
  (7, 'Prospecting'),
  (8, 'Behavior'),
  (9, 'Interest'),
  (10, 'TikTok'),
  (11, 'Feed'),
  (12, 'Android'),
  (13, 'Multi'),
  (14, 'Multi'),
  (15, 'Social'),
  (16, 'NA'),
  (17, 'Android'),
  (18, 'DSS'),
  (19, 'Disney+'),
  (20, 'KR'),
  (21, '')],
 22)

In [442]:
for prop, name in zip(convList, nameList):
    t= "".join([" "]*(42-len(prop[1])-len(str(prop[0]))))
    print(f"{prop[0]}: {prop[1]}{t} -> {name[1]}")

0: Disney+ Targeting Geo (country)           -> KR
1: Disney+ Language (Auto-tag)               -> KO
2: Disney+ Brand (Auto-tag)                  -> Multi
3: Disney+ Title (Auto-tag)                  -> Rotation
4: Disney+ Campaign Objective (Auto-tag)     -> Consid
5: Disney+ Content Type (Auto-tag)           -> Rotation
6: Disney+ Quarter / Timing (Auto-tag)       -> 23'Q3
7: Disney+ Targeting Strategy (Auto-tag)     -> Prospecting
8: Disney+ Targeting Type (Auto-tag)         -> Behavior
9: Disney+ Audience Detail (Auto-tag)        -> Interest
10: Disney+ Platform/Vendor Name (Auto-tag)  -> TikTok
11: Disney+ Placement (Auto-tag)             -> Feed
12: Disney+ Device (Auto-tag)                -> Android
13: Disney+ Ad Format (Auto-tag)             -> Multi
14: Disney+ Unit Size (Auto-tag)             -> Multi
15: Disney+ Channel (Internal)               -> Social
16: Disney+ Creative (Auto-tag)              -> NA
17: Disney+ Detail FreeForm (Auto-tag)       -> Android
18: Disney+ C

In [421]:
dic = {}
x = dic.get(0, [])
x.append(1)
x, dic

([1], {})

In [422]:
x = [1]
y = [2]
x.append(y)
x

[1, [2]]

In [433]:
len("Disney+ Detail FreeForm (Auto-tag)")

34